In [88]:
import pandas as pd
import numpy as np

In [89]:
data = pd.read_csv('./merged_market_data.csv')
data

,tickerSymbol,exchangeSymbol,ISOCode,priceClose,pricingDate
0,ALB,NYSE,USD,162.08,2023-10-10
1,ALB,NYSE,USD,155.75,2023-10-09
2,ALB,NYSE,USD,159.84,2023-10-06
3,ALB,NYSE,USD,155.03,2023-10-05
4,ALB,NYSE,USD,158.40,2023-10-04
...,...,...,...,...,...
846007,VLTO,NYSE,USD,77.05,2023-10-03
846008,VLTO,NYSE,USD,85.12,2023-10-02
846009,VLTO,NYSE,USD,84.56,2023-09-29
846010,VLTO,NYSE,USD,84.65,2023-09-28


In [90]:
data['ISOCode'].unique()

array(['USD'], dtype=object)

In [91]:
# in-sample : start="2018-05-01", end="2023-04-30"
# out-sample : start = '2023-05-01', end = '2023-10-31'

In [92]:
max(data['pricingDate'].unique())

'2023-10-10'

In [93]:
len(data['tickerSymbol'].unique())

499

### 공분산 행렬

In [94]:
# Let's create the pivot table again with the actual ticker symbols present in the data

# Pivot the table to get dates as rows and tickers as columns with priceClose as values
# This time we will not fill missing tickers with NaN or create a range of tickers
# because we want to use the actual ticker symbols present in the data
actual_price_matrix = data.pivot_table(index='pricingDate', columns='tickerSymbol', values='priceClose', aggfunc='mean')

# Check the shape of the actual pivot table and show the head to confirm non-NaN values
actual_price_matrix_shape = actual_price_matrix.shape
actual_price_matrix_head = actual_price_matrix.head()

actual_price_matrix_shape, actual_price_matrix_head


((1939, 499),
 tickerSymbol      A    AAL     AAPL   ABBV  ABNB    ABT       ACGL     ACN  \
 pricingDate                                                                  
 2017-01-03    46.49  46.30  29.0375  62.41   NaN  39.05  28.630000  116.46   
 2017-01-04    47.10  46.70  29.0050  63.29   NaN  39.36  28.833333  116.74   
 2017-01-05    46.54  45.89  29.1525  63.77   NaN  39.70  28.540000  114.99   
 2017-01-06    47.99  46.21  29.4775  63.79   NaN  40.78  28.823333  116.30   
 2017-01-09    48.14  47.08  29.7475  64.21   NaN  40.74  28.406666  115.00   
 
 tickerSymbol    ADBE     ADI  ...   WYNN    XEL    XOM   XRAY    XYL    YUM  \
 pricingDate                   ...                                             
 2017-01-03    103.48  72.505  ...  87.46  40.62  90.89  58.62  49.65  63.21   
 2017-01-04    104.14  72.360  ...  90.28  40.80  89.89  59.10  50.39  63.44   
 2017-01-05    105.91  71.320  ...  91.44  40.80  88.55  58.33  49.93  63.65   
 2017-01-06    108.30  71.600  

In [95]:
actual_price_matrix

tickerSymbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2017-01-03,46.49,46.30,29.0375,62.41,NaN,39.05,28.630000,116.46,103.48,72.505,...,87.46,40.62,90.89,58.62,49.65,63.21,103.33,86.25,43.18,53.59
2017-01-04,47.10,46.70,29.0050,63.29,NaN,39.36,28.833333,116.74,104.14,72.360,...,90.28,40.80,89.89,59.10,50.39,63.44,104.28,87.03,43.80,54.11
2017-01-05,46.54,45.89,29.1525,63.77,NaN,39.70,28.540000,114.99,105.91,71.320,...,91.44,40.80,88.55,58.33,49.93,63.65,104.95,84.75,43.09,53.93
2017-01-06,47.99,46.21,29.4775,63.79,NaN,40.78,28.823333,116.30,108.30,71.600,...,92.43,40.92,88.50,58.29,49.58,64.42,104.96,85.96,43.37,54.10
2017-01-09,48.14,47.08,29.7475,64.21,NaN,40.74,28.406666,115.00,108.57,71.940,...,92.75,40.30,87.04,58.47,49.37,64.60,107.00,85.97,42.90,53.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.67,12.73,173.6600,147.69,127.41,95.65,80.460000,309.39,518.42,173.790,...,88.21,56.63,111.50,33.30,91.22,123.53,109.69,230.94,33.33,170.99
2023-10-05,110.35,12.85,174.9100,147.45,124.99,96.20,81.640000,309.66,516.44,172.100,...,89.49,56.48,108.99,32.18,90.14,121.84,109.30,222.54,33.40,172.00
2023-10-06,110.64,12.76,177.4900,148.24,126.36,96.88,82.180000,312.19,526.68,173.970,...,92.41,57.35,107.17,32.48,90.97,119.46,110.91,223.85,33.69,175.58


In [96]:
sp500_in = actual_price_matrix.loc['2018-05-01':'2023-04-30']
sp500_out = actual_price_matrix.loc['2023-05-01':]

In [97]:
sp500_in

tickerSymbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,66.24,43.36,42.2750,102.07,NaN,58.82,26.876666,151.92,224.08,88.86,...,191.85,46.65,76.95,50.64,70.37,86.64,116.95,136.78,55.05,83.40
2018-05-02,65.91,42.24,44.1425,100.37,NaN,57.85,26.310000,150.71,221.10,88.30,...,191.03,46.54,76.80,50.59,68.88,80.20,117.46,133.12,55.19,79.90
2018-05-03,66.34,42.11,44.2225,100.28,NaN,57.93,25.983333,151.73,226.05,87.91,...,191.62,46.48,76.54,49.24,69.63,82.67,115.18,133.54,54.45,80.38
2018-05-04,67.00,42.86,45.9575,100.17,NaN,58.74,26.276666,153.44,228.51,90.27,...,192.41,46.39,76.90,49.99,70.08,82.43,116.22,134.67,55.46,81.56
2018-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24,138.48,13.32,165.3300,164.08,116.120,110.40,72.510000,277.25,377.34,186.96,...,114.48,71.03,118.20,41.44,104.54,138.90,138.14,288.19,28.27,176.35
2023-04-25,130.37,12.84,163.7700,164.90,112.505,109.97,72.590000,270.70,369.59,181.14,...,111.63,71.18,116.52,40.97,102.92,137.81,139.06,284.10,26.73,173.02
2023-04-26,133.12,12.74,163.7600,161.80,113.680,108.75,72.290000,271.21,363.06,180.66,...,109.45,69.68,115.45,40.93,100.98,138.01,139.08,280.42,26.39,172.94


In [98]:
sp500_out

tickerSymbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2023-05-01,136.08,13.89,169.59,153.71,121.42,111.11,75.90,279.42,374.15,182.54,...,115.60,70.17,114.67,42.39,104.50,142.23,138.83,287.45,26.82,179.56
2023-05-02,134.52,13.77,168.54,151.62,120.58,111.50,76.03,276.74,368.66,183.66,...,113.20,69.10,110.10,42.10,104.52,142.90,143.01,255.44,23.92,176.90
2023-05-03,134.77,13.79,167.45,149.25,118.86,111.76,75.62,275.18,345.25,182.39,...,109.33,68.50,107.93,40.48,104.15,137.31,139.62,258.50,22.66,177.73
2023-05-04,133.88,13.51,165.79,147.36,117.63,110.86,72.92,266.50,335.83,181.33,...,109.25,69.11,106.04,40.85,106.33,136.32,136.63,265.00,19.93,178.47
2023-05-05,133.30,13.87,173.57,148.03,119.90,111.26,74.81,265.65,348.40,183.39,...,110.78,69.57,108.68,41.30,108.97,137.49,137.91,273.80,23.76,186.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.67,12.73,173.66,147.69,127.41,95.65,80.46,309.39,518.42,173.79,...,88.21,56.63,111.50,33.30,91.22,123.53,109.69,230.94,33.33,170.99
2023-10-05,110.35,12.85,174.91,147.45,124.99,96.20,81.64,309.66,516.44,172.10,...,89.49,56.48,108.99,32.18,90.14,121.84,109.30,222.54,33.40,172.00
2023-10-06,110.64,12.76,177.49,148.24,126.36,96.88,82.18,312.19,526.68,173.97,...,92.41,57.35,107.17,32.48,90.97,119.46,110.91,223.85,33.69,175.58


In [99]:
# covariance matrix (sp500_in)
cov_matrix = sp500_in.cov()
cov_matrix

tickerSymbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
tickerSymbol,,,,,,,,,,,,,,,,,,,,,
A,1102.296971,-192.380208,1408.325018,713.086083,44.643125,605.158588,227.440923,2016.266831,3074.654547,978.610674,...,-434.818851,184.601002,122.112874,46.158861,558.542786,449.167827,186.299867,3668.371182,149.760478,1162.200494
AAL,-192.380208,85.620200,-311.627076,-136.518349,73.048192,-132.220539,-41.342584,-358.543795,-605.695935,-173.189207,...,223.185157,-60.647364,29.624568,15.320114,-63.412549,-61.619429,-11.874997,-540.978668,21.178732,-251.490609
AAPL,1408.325018,-311.627076,2090.854933,1077.522948,-7.213235,837.447577,308.736134,2784.375856,4015.393698,1311.913200,...,-764.994001,283.297405,152.647016,7.961210,629.939878,560.366190,172.436183,4639.440435,176.253202,1632.734250
ABBV,713.086083,-136.518349,1077.522948,800.082847,-439.693790,342.618272,226.777103,1327.249244,1077.489184,690.518444,...,-360.736562,120.984240,344.787123,-90.343842,263.509316,300.808866,-56.884322,1604.063867,121.260611,668.901290
ABNB,44.643125,73.048192,-7.213235,-439.693790,997.900791,224.059457,-192.401454,434.296549,2389.905475,24.305546,...,357.234595,-49.080031,-544.054402,303.141189,163.513769,-47.700327,391.962682,2820.611094,129.716487,360.826954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,449.167827,-61.619429,560.366190,300.808866,-47.700327,248.221154,137.567790,870.696963,1058.711218,441.547050,...,-142.679392,86.263371,131.755554,35.539415,243.331622,265.857224,84.513365,1441.267112,75.470732,487.784271
ZBH,186.299867,-11.874997,172.436183,-56.884322,391.962682,156.990023,10.719986,330.641706,1156.855926,172.770401,...,138.875729,47.119468,-155.228145,134.641130,171.829121,84.513365,313.371565,1265.407548,17.186504,264.919216
ZBRA,3668.371182,-540.978668,4639.440435,1604.063867,2820.611094,2393.861979,398.656962,7513.257412,14508.690656,3247.960338,...,-1007.276502,597.719025,-586.587796,701.437212,2097.222537,1441.267112,1265.407548,17199.023195,692.448811,4595.058264


### Simple EPO 구현 - input : cov_matrix, corr_matrix 필요

In [37]:
pip install cvxpy

  Obtaining dependency information for cvxpy from https://files.pythonhosted.org/packages/fc/56/377e9412449a48340ee070eee16ed205b7c4170c36bd1b67345898da21e4/cvxpy-1.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for osqp>=0.6.2 from https://files.pythonhosted.org/packages/23/46/232041be3d85624ec4c20b3bdb8bd4bfb6640851901db155917e0c412aef/osqp-0.6.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/72.0 kB ? eta -:--:--
     ---------------------------------------- 72.0/72.0 kB 4.1 MB/s eta 0:00:00
  Obtaining dependency information for clarabel>=0.5.0 from https://files.pythonhosted.org/packages/a8/6b/0f04f31c1528ab1f6bbc9e6ff83d019386941db4c9e0a7ddd7d16f5a85d2/clarabel-0.6.0-cp37-abi3-win_amd64.whl.metadata
  Obtaining dependency information for scs>=3.0 from https://files.pythonhosted.org/packages/12/d1/9d1d653e1b70455d1a427b9208c0f9ceb02092d955575da7bc5c95489592/scs-3.2.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dep

In [83]:
# import cvxpy as cp

In [84]:
# from cvxpy.atoms.affine.wraps import psd_wrap
# cov_matrix_psd_wrapped = psd_wrap(cov_matrix_psd)

In [100]:
# 가중치 제약
n_assets = cov_matrix.shape[0]
min_weight = (1/2) * (1/n_assets)
max_weight = 2 * (1/n_assets)

### 모든 종목에 대해 opt

In [101]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function (portfolio variance)
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

# 상관관계 축소를 위한 λ 값 설정 (예시로 0.5를 사용)
lambda_shrinkage = 0.5

# Convert DataFrame to NumPy array and ensure it's symmetric
cov_matrix_np = cov.to_numpy()
cov_matrix_symmetric = (cov_matrix_np + cov_matrix_np.T) / 2

# 상관관계 행렬 조정
corr_matrix = np.corrcoef(cov_matrix_symmetric)  # 상관관계 행렬 계산
identity_matrix = np.eye(corr_matrix.shape[0])
adjusted_corr_matrix = (1 - lambda_shrinkage) * corr_matrix + lambda_shrinkage * identity_matrix

# 각 자산의 변동성 계산
volatilities = np.sqrt(np.diag(cov_matrix_symmetric))

# 분산-공분산 행렬 재계산
cov_matrix_psd = np.outer(volatilities, volatilities) * adjusted_corr_matrix

# Constraints and bounds
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = [(min_weight, max_weight) for _ in range(n_assets)]

# Initial guess
initial_guess = [1/n_assets] * n_assets

# Perform the optimization
result = minimize(portfolio_variance, initial_guess, args=(cov_matrix_psd,), method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the ticker symbols from the covariance matrix columns
ticker_symbols = cov.columns

# Extract the optimal weights
optimal_weights = result.x
optimal_weight_dict = {ticker: weight for ticker, weight in zip(ticker_symbols, optimal_weights)}

In [102]:
optimal_weight_dict

{'A': 0.002004008016032064,
 'AAL': 0.002004008016032064,
 'AAPL': 0.002004008016032064,
 'ABBV': 0.002004008016032064,
 'ABNB': 0.002004008016032064,
 'ABT': 0.002004008016032064,
 'ACGL': 0.002004008016032064,
 'ACN': 0.002004008016032064,
 'ADBE': 0.002004008016032064,
 'ADI': 0.002004008016032064,
 'ADM': 0.002004008016032064,
 'ADP': 0.002004008016032064,
 'ADSK': 0.002004008016032064,
 'AEE': 0.002004008016032064,
 'AEP': 0.002004008016032064,
 'AES': 0.002004008016032064,
 'AFL': 0.002004008016032064,
 'AIG': 0.002004008016032064,
 'AIZ': 0.002004008016032064,
 'AJG': 0.002004008016032064,
 'AKAM': 0.002004008016032064,
 'ALB': 0.002004008016032064,
 'ALGN': 0.002004008016032064,
 'ALK': 0.002004008016032064,
 'ALL': 0.002004008016032064,
 'ALLE': 0.002004008016032064,
 'AMAT': 0.002004008016032064,
 'AMCR': 0.002004008016032064,
 'AMD': 0.002004008016032064,
 'AME': 0.002004008016032064,
 'AMGN': 0.002004008016032064,
 'AMP': 0.002004008016032064,
 'AMT': 0.002004008016032064,
